# TP2- Modelo con LightGBM

## Set up inicial

In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

## Carga de los datasets y info

Se cargan las columnas con formatos adecuados según el tipo de datos. Se usa como índice building_id

In [2]:
# Train values
# Levantar los datos de cada edificio/casa
df = pd.read_csv('datos/train_values.csv',nrows=0)
columnas = df.columns.to_list()
datatypes = {
    **{columnas[0]:np.int32}\
    , **{column:np.int16 for column in columnas[1:8] + columnas[27:28]}\
    , **{column:'category' for column in columnas[8:15] + columnas[26:27]}\
    , **{column:np.bool for column in columnas[15:26] + columnas[28:39]}
}
train_values = pd.read_csv('datos/train_values.csv',dtype=datatypes)
train_values = train_values.set_index("building_id")
columnas = train_values.columns.to_list()

In [3]:
#Train labels
train_labels = pd.read_csv('datos/train_labels.csv',index_col='building_id')

LightGBM no requiere hacer OneHotEncoding..

In [5]:
import lightgbm as lgb

In [13]:
model = lgb.LGBMClassifier(task='train',objective='multiclass',boosting_type='gbdt',seed=99)
model

LGBMClassifier(objective='multiclass', seed=99, task='train')

In [14]:
model.fit(train_values, train_labels.values.ravel() )

LGBMClassifier(objective='multiclass', seed=99, task='train')

In [15]:
from sklearn.metrics import f1_score
y_pred = model.predict(train_values)
f1_score(train_labels, y_pred, average='micro')

0.7182934831408937

Submit de prueba

In [19]:
test_values = pd.read_csv('datos/test_values.csv', index_col='building_id',dtype=datatypes)
predictions = model.predict(test_values)

In [20]:
submission_format = pd.read_csv('datos/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submissions/lightgbm_submission.csv')

In [21]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3
